### Create SPARK_HOME and PYLIB env var and update PATH env var

In [1]:
import os
import sys
os.environ["SPARK_HOME"] = "/usr/hdp/current/spark2-client"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
sys.path.insert(0, os.environ["PYLIB"] + "/py4j-0.10.4-src.zip")
sys.path.insert(0, os.environ["PYLIB"] + "/pyspark.zip")

### Initializing Spark

Build __SparkConf__ object 

    Contains information about your application.  


Create __SparkContext__ object 
    
    Tells Spark how to access a cluster. 
    

Create __SparkSession__ object

    The entry point to programming Spark with the Dataset and DataFrame API.

    Used to create DataFrame, register DataFrame as tables and execute SQL over tables etc.

In [2]:
from pyspark.conf import SparkConf
from pyspark import SparkContext
from pyspark.sql import SparkSession

conf = SparkConf().setAppName("Universal Bank Data Set").setMaster('local')
sc = SparkContext(conf=conf)
spark = SparkSession(sc)

### Loading the dependent libraries

In [3]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
# from pyspark.sql.functions import isnan, when, count, col, countDistinct


#### Problem Statement
The dataset is from a bank, data related to direct marketing campaigns of a Portuguese banking institution. The marketing campaigns were based on phone calls. Often, more than one contact to the same client was required, to access if the product (bank term deposit) would be (or not) subscribed. The data and attribute description are in the folder. 


#### Data Dictionary
 The dataset has the following attributes:

1 - age (numeric)

2 - job : type of job (categorical: "admin.","unknown","unemployed","management","housemaid","entrepreneur","student",
                                    "blue-collar","self-employed","retired","technician","services") 

3 - marital : marital status (categorical: "married","divorced","single"; note: "divorced" means divorced or widowed)

4 - education (categorical: "unknown","secondary","primary","tertiary")

5 - default: has credit in default? (binary: "yes","no")

#### 6 - balance: average yearly balance, in euros (numeric) 

7 - housing: has housing loan? (binary: "yes","no")

8 - loan: has personal loan? (binary: "yes","no")

9 - contact: contact communication type (categorical: "unknown","telephone","cellular") 

10 - day: last contact day of the month (numeric)

11 - month: last contact month of year (categorical: "jan", "feb", "mar", ..., "nov", "dec")

12 - duration: last contact duration, in seconds (numeric)

13 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)

14 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric, -1 means client was not previously contacted)
  
15 - previous: number of contacts performed before this campaign and for this client (numeric)

16 - poutcome: outcome of the previous marketing campaign (categorical: "unknown","other","failure","success")

17 - Approved_no_yes - has the client subscribed to a __term deposit?__ (binary: "yes","no")

### Defining the schema to data

In [4]:
## Define Schema
bankDataSchema = StructType([
    StructField("age", IntegerType(), True),
    StructField("job", StringType(), True),
    StructField("marital_status", StringType(), True),
    StructField("education", StringType(), True),
    StructField("default", StringType(), True),
    StructField("balance", DoubleType(), True),
    StructField("housing", StringType(), True),
    StructField("loan", StringType(), True),        
    StructField("contact", StringType(), True),
    StructField("day", IntegerType(), True),
    StructField("month", StringType(), True),
    StructField("duration", DoubleType(), True),
    StructField("campaign", DoubleType(), True),
    StructField("pdays", DoubleType(), True),
    StructField("previous", DoubleType(), True),
    StructField("poutcome", StringType(), True),
    StructField("Approved_no_yes", StringType(), True)])

### Reading the data and creating a dataframe

In [5]:
## Read data and create a dataframe
data = spark.read.format("csv")\
       .option("header", "false")\
       .option("inferSchema", "true")\
       .load("hdfs:///user/jeevans/bankdata/", schema = bankDataSchema)

### Understanding Data

#### Print Schema

In [6]:
data.printSchema()

root
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital_status: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: double (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- campaign: double (nullable = true)
 |-- pdays: double (nullable = true)
 |-- previous: double (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- Approved_no_yes: string (nullable = true)



#### Another way to check the data type of each attribute

In [7]:
data.dtypes

[('age', 'int'),
 ('job', 'string'),
 ('marital_status', 'string'),
 ('education', 'string'),
 ('default', 'string'),
 ('balance', 'double'),
 ('housing', 'string'),
 ('loan', 'string'),
 ('contact', 'string'),
 ('day', 'int'),
 ('month', 'string'),
 ('duration', 'double'),
 ('campaign', 'double'),
 ('pdays', 'double'),
 ('previous', 'double'),
 ('poutcome', 'string'),
 ('Approved_no_yes', 'string')]

#### Total number of Columns and Records

In [8]:
print("No. of Columns = {}".format(len(data.columns)))

print('No. of Records = {}'.format(data.count()))

No. of Columns = 17
No. of Records = 4521


#### Look at first 3 row of the dataframe

In [9]:
data.head(3)

[Row(age=30, job=u'unemployed', marital_status=u'married', education=u'primary', default=u'no', balance=1787.0, housing=u'no', loan=u'no', contact=u'cellular', day=19, month=u'oct', duration=79.0, campaign=1.0, pdays=-1.0, previous=0.0, poutcome=u'unknown', Approved_no_yes=u'no'),
 Row(age=33, job=u'services', marital_status=u'married', education=u'secondary', default=u'no', balance=4789.0, housing=u'yes', loan=u'yes', contact=u'cellular', day=11, month=u'may', duration=220.0, campaign=1.0, pdays=339.0, previous=4.0, poutcome=u'failure', Approved_no_yes=u'no'),
 Row(age=35, job=u'management', marital_status=u'single', education=u'tertiary', default=u'no', balance=1350.0, housing=u'yes', loan=u'no', contact=u'cellular', day=16, month=u'apr', duration=185.0, campaign=1.0, pdays=330.0, previous=1.0, poutcome=u'failure', Approved_no_yes=u'no')]

In [10]:
data.show(3)

+---+----------+--------------+---------+-------+-------+-------+----+--------+---+-----+--------+--------+-----+--------+--------+---------------+
|age|       job|marital_status|education|default|balance|housing|loan| contact|day|month|duration|campaign|pdays|previous|poutcome|Approved_no_yes|
+---+----------+--------------+---------+-------+-------+-------+----+--------+---+-----+--------+--------+-----+--------+--------+---------------+
| 30|unemployed|       married|  primary|     no| 1787.0|     no|  no|cellular| 19|  oct|    79.0|     1.0| -1.0|     0.0| unknown|             no|
| 33|  services|       married|secondary|     no| 4789.0|    yes| yes|cellular| 11|  may|   220.0|     1.0|339.0|     4.0| failure|             no|
| 35|management|        single| tertiary|     no| 1350.0|    yes|  no|cellular| 16|  apr|   185.0|     1.0|330.0|     1.0| failure|             no|
+---+----------+--------------+---------+-------+-------+-------+----+--------+---+-----+--------+--------+-----

#### Summary statistics

In [11]:
data.describe().show()

+-------+------------------+-------+--------------+---------+-------+------------------+-------+----+--------+------------------+-----+------------------+------------------+------------------+------------------+--------+---------------+
|summary|               age|    job|marital_status|education|default|           balance|housing|loan| contact|               day|month|          duration|          campaign|             pdays|          previous|poutcome|Approved_no_yes|
+-------+------------------+-------+--------------+---------+-------+------------------+-------+----+--------+------------------+-----+------------------+------------------+------------------+------------------+--------+---------------+
|  count|              4521|   4521|          4521|     4521|   4521|              4521|   4521|4521|    4521|              4521| 4521|              4521|              4521|              4521|              4521|    4521|           4521|
|   mean| 41.17009511170095|   null|          null| 

Show only fixed set of colums 

In [12]:
data.describe().select('summary', 'age', 'loan', 'balance', 'pdays').show()

+-------+------------------+----+------------------+------------------+
|summary|               age|loan|           balance|             pdays|
+-------+------------------+----+------------------+------------------+
|  count|              4521|4521|              4521|              4521|
|   mean| 41.17009511170095|null|1422.6578190665782|39.766644547666445|
| stddev|10.576210958711263|null|3009.6381424673395|100.12112444301656|
|    min|                19|  no|           -3313.0|              -1.0|
|    max|                87| yes|           71188.0|             871.0|
+-------+------------------+----+------------------+------------------+



Observation

    Balance has -ve values

In [13]:
data.where(data.balance < 0).count()

366

### Data Preprocessing

#### Replace negative balances with zeroes

In [14]:
data = data.withColumn('balance', when(data.balance > 0, data.balance).otherwise(0))

#### Handling missing values

Checking for null values at each column

In [15]:
data.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in data.columns]).show()

+---+---+--------------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---------------+
|age|job|marital_status|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|Approved_no_yes|
+---+---+--------------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---------------+
|  0|  0|             0|        0|      0|      0|      0|   0|      0|  0|    0|       0|       0|    0|       0|       0|              0|
+---+---+--------------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---------------+



#### Split the data into training and test sets (30% held out for testing)

In [16]:
(trainingData, testData) = data.randomSplit([0.7, 0.3])

#### Creating a list of categorical and numerical features

In [17]:
cat_Var_Names = ['job', 'marital_status', 'education', 'default', 'housing', 
                 'loan', 'day', 'contact', 'month', 'poutcome', 'Approved_no_yes']

num_Var_Names = ['age', 'duration', 'previous', 'pdays', 'campaign']

#### Use VectorAssembler to combine a given list of numcolumns into a single vector column.

In [18]:
from pyspark.ml.feature import VectorAssembler

assembler_Num = VectorAssembler(inputCols=num_Var_Names, outputCol="num_features")

#### Scale all the numeric attributes using MinMaxScaler

    MinMaxScaler transforms a dataset of Vector rows, rescaling each feature to a specific range (often [0, 1]). 

    MinMaxScaler computes summary statistics on a data set and produces a MinMaxScalerModel. The model can then transform each feature individually such that it is in the given range.

In [19]:
from pyspark.ml.feature import MinMaxScaler

min_Max_Scalar = MinMaxScaler(inputCol="num_features", outputCol="scaled_num_features")

#### Covert categorical to numeric : 

    OneHotEncoder, StringIndexer, VectorAssembler,  VectorIndexer

In [20]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler

indexers_Cat = [StringIndexer(inputCol=cat_Var_Name, outputCol="{0}_index".format(cat_Var_Name)) for cat_Var_Name in cat_Var_Names ]
encoders_Cat = [OneHotEncoder(inputCol=indexer.getOutputCol(), outputCol="{0}_vec".format(indexer.getInputCol())) for indexer in indexers_Cat]
assembler_Cat = VectorAssembler(inputCols=[encoder.getOutputCol() for encoder in encoders_Cat], outputCol="cat_features")

assembler = VectorAssembler(inputCols=["scaled_num_features", "cat_features"], outputCol="features")


In [21]:
preprocessiong_Stages = [assembler_Num]+[min_Max_Scalar]+indexers_Cat+encoders_Cat+[assembler_Cat]+[assembler]

### Model Building and Evaluation

#### Logistic Regression

In [22]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(maxIter=10, labelCol="balance", featuresCol="features")

In [23]:
from pyspark.ml import Pipeline

lr_Pipeline = Pipeline(stages=preprocessiong_Stages+[lr]) 

lr_Pipeline_model = lr_Pipeline.fit(trainingData)

In [24]:
print("Coefficients: " + str(lr_Pipeline_model.stages[-1].coefficients))
print("Intercept: " + str(lr_Pipeline_model.stages[-1].intercept))

Coefficients: [868.874044464,-441.108182646,522.687594496,-104.289169877,-497.336053763,515.008241408,2.89641699949,205.307369835,40.5057608004,92.3141144337,824.009277885,77.8209084445,632.989081613,23.1442099924,894.581526324,-107.089425044,360.247598293,477.839801578,-369.33013256,-180.35338757,-303.427839034,1208.51392942,-114.484803343,408.113940147,783.386976751,899.766534576,709.477855079,317.519297351,1033.5948376,69.7654398607,1059.35179227,713.739360509,675.513275397,881.834605104,1207.15950729,176.342058092,553.424348835,373.82871893,458.910801253,569.138820696,899.19730639,769.059600712,295.921796401,1000.99759171,606.222994592,551.271101529,219.896776202,1014.42335449,702.667435941,1122.97911173,-108.70090842,1485.31218723,179.804345479,575.673947405,-92.4701750924,-124.072427577,-2787.55193241,-3276.76242128,-2718.07233186,-2279.14673302,-1740.82029764,-2486.91196124,-2795.66734152,-3078.26970129,-916.496852529,-2358.37648362,-2549.49812436,25.4619280769,-19.3183900288,-6

In [25]:
train_predictions_lr = lr_Pipeline_model.transform(trainingData)
test_predictions_lr = lr_Pipeline_model.transform(testData)

In [26]:
test_predictions_lr.show(2)

+---+--------+--------------+---------+-------+-------+-------+----+---------+---+-----+--------+--------+-----+--------+--------+---------------+--------------------+--------------------+---------+--------------------+---------------+-------------+-------------+----------+---------+-------------+-----------+--------------+---------------------+---------------+------------------+-------------+-------------+-----------+-------------+---------------+-------------+--------------+-------------+-------------------+--------------------+--------------------+------------------+
|age|     job|marital_status|education|default|balance|housing|loan|  contact|day|month|duration|campaign|pdays|previous|poutcome|Approved_no_yes|        num_features| scaled_num_features|job_index|marital_status_index|education_index|default_index|housing_index|loan_index|day_index|contact_index|month_index|poutcome_index|Approved_no_yes_index|        job_vec|marital_status_vec|education_vec|  default_vec|housing_vec| 

#### Evaluation : LR Model

In [27]:
# Find the error metric - RMSE
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(labelCol="balance", predictionCol="prediction", metricName="rmse" )

train_rmse_lr = evaluator.evaluate(train_predictions_lr)
print("Train RMSE  = " + str(train_rmse_lr))

test_rmse_lr = evaluator.evaluate(test_predictions_lr)
print("Test RMSE = " + str(test_rmse_lr))

Train RMSE  = 2974.83648786
Test RMSE = 2729.75583955


#### Decision Tree Regressor

In [28]:
from pyspark.ml.regression import DecisionTreeRegressor

dt = DecisionTreeRegressor(labelCol="balance", featuresCol="features")

In [29]:
dt_Pipeline = Pipeline(stages=preprocessiong_Stages+[dt]) 

dt_Pipeline_model = dt_Pipeline.fit(trainingData)

In [30]:
train_predictions_dt = dt_Pipeline_model.transform(trainingData)
test_predictions_dt = dt_Pipeline_model.transform(testData)

In [31]:
test_predictions_dt.show(2)

+---+--------+--------------+---------+-------+-------+-------+----+---------+---+-----+--------+--------+-----+--------+--------+---------------+--------------------+--------------------+---------+--------------------+---------------+-------------+-------------+----------+---------+-------------+-----------+--------------+---------------------+---------------+------------------+-------------+-------------+-----------+-------------+---------------+-------------+--------------+-------------+-------------------+--------------------+--------------------+-----------------+
|age|     job|marital_status|education|default|balance|housing|loan|  contact|day|month|duration|campaign|pdays|previous|poutcome|Approved_no_yes|        num_features| scaled_num_features|job_index|marital_status_index|education_index|default_index|housing_index|loan_index|day_index|contact_index|month_index|poutcome_index|Approved_no_yes_index|        job_vec|marital_status_vec|education_vec|  default_vec|housing_vec|  

#### Evaluation : DT Model

In [32]:
train_rmse_dt = evaluator.evaluate(train_predictions_dt)
print("Train RMSE  = " + str(train_rmse_dt))

test_rmse_dt = evaluator.evaluate(test_predictions_dt)
print("Test RMSE = " + str(test_rmse_dt))

Train RMSE  = 2679.17101032
Test RMSE = 3631.36617782


#### Random Forest Classifier

In [33]:
from pyspark.ml.regression import RandomForestRegressor

rf = RandomForestRegressor(labelCol="balance", featuresCol="features", numTrees=10)

In [34]:
rf_Pipeline = Pipeline(stages=preprocessiong_Stages+[rf]) 

rf_Pipeline_model = rf_Pipeline.fit(trainingData)

In [35]:
train_predictions_rf = rf_Pipeline_model.transform(trainingData)
test_predictions_rf = rf_Pipeline_model.transform(testData)

In [36]:
test_predictions_rf.show(2)

+---+--------+--------------+---------+-------+-------+-------+----+---------+---+-----+--------+--------+-----+--------+--------+---------------+--------------------+--------------------+---------+--------------------+---------------+-------------+-------------+----------+---------+-------------+-----------+--------------+---------------------+---------------+------------------+-------------+-------------+-----------+-------------+---------------+-------------+--------------+-------------+-------------------+--------------------+--------------------+------------------+
|age|     job|marital_status|education|default|balance|housing|loan|  contact|day|month|duration|campaign|pdays|previous|poutcome|Approved_no_yes|        num_features| scaled_num_features|job_index|marital_status_index|education_index|default_index|housing_index|loan_index|day_index|contact_index|month_index|poutcome_index|Approved_no_yes_index|        job_vec|marital_status_vec|education_vec|  default_vec|housing_vec| 

#### Evaluation : RF Model

In [37]:
train_rmse_rf = evaluator.evaluate(train_predictions_rf)
print("Train RMSE  = " + str(train_rmse_rf))

test_rmse_rf = evaluator.evaluate(test_predictions_rf)
print("Test RMSE = " + str(test_rmse_rf))

Train RMSE  = 2672.8105754
Test RMSE = 2740.71203683


#### Gradient Boosted Tree Classifier

In [38]:
from pyspark.ml.regression import GBTRegressor

gbt = GBTRegressor(labelCol="balance", featuresCol="features", maxIter=10)

In [39]:
gbt_Pipeline = Pipeline(stages=preprocessiong_Stages+[gbt]) 

gbt_Pipeline_model = gbt_Pipeline.fit(trainingData)

In [40]:
train_predictions_gbt = gbt_Pipeline_model.transform(trainingData)
test_predictions_gbt = gbt_Pipeline_model.transform(testData)

In [41]:
test_predictions_gbt.show(2)

+---+--------+--------------+---------+-------+-------+-------+----+---------+---+-----+--------+--------+-----+--------+--------+---------------+--------------------+--------------------+---------+--------------------+---------------+-------------+-------------+----------+---------+-------------+-----------+--------------+---------------------+---------------+------------------+-------------+-------------+-----------+-------------+---------------+-------------+--------------+-------------+-------------------+--------------------+--------------------+------------------+
|age|     job|marital_status|education|default|balance|housing|loan|  contact|day|month|duration|campaign|pdays|previous|poutcome|Approved_no_yes|        num_features| scaled_num_features|job_index|marital_status_index|education_index|default_index|housing_index|loan_index|day_index|contact_index|month_index|poutcome_index|Approved_no_yes_index|        job_vec|marital_status_vec|education_vec|  default_vec|housing_vec| 

#### Evaluation : gbt Model

In [42]:
train_rmse_gbt = evaluator.evaluate(train_predictions_gbt)

print("Train RMSE  = " + str(train_rmse_gbt))

test_rmse_gbt = evaluator.evaluate(test_predictions_gbt)

print("Test RMSE = " + str(test_rmse_gbt))

Train RMSE  = 2393.73615068
Test RMSE = 3677.13057598
